In [10]:
import requests
import pandas as pd
from datetime import datetime

# Base URL
base_url = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Daily_Trade_Data/FeatureServer/0/query"

# ArcGIS date filter
start_date = "2022-01-01"
today_date = datetime.today().strftime("%Y-%m-%d")

params = {
    "where": f"date >= DATE '{start_date}' AND date <= DATE '{today_date}'",
    "outFields": "*",
    "outSR": "4326",
    "f": "json",
    "returnGeometry": "false",
    "resultOffset": 0,
    "resultRecordCount": 2000
}

all_data = []

# Fetch data in batches
while True:
    response = requests.get(base_url, params=params)
    data = response.json()
    features = data.get("features", [])
    if not features:
        break
    all_data.extend([f["attributes"] for f in features])
    print(f"Fetched {len(all_data)} records so far...")
    params["resultOffset"] += params["resultRecordCount"]

# Load into DataFrame
df = pd.DataFrame(all_data)

# Convert epoch to readable date
df['timestamp'] = pd.to_datetime(df['date'], unit='ms').dt.date

# Define metrics to aggregate
numeric_cols = [
    'portcalls_container', 'portcalls_dry_bulk', 'portcalls_general_cargo',
    'portcalls_roro', 'portcalls_tanker', 'portcalls_cargo', 'portcalls',
    'import_container', 'import_dry_bulk', 'import_general_cargo',
    'import_roro', 'import_tanker', 'import_cargo', 'import',
    'export_container', 'export_dry_bulk', 'export_general_cargo',
    'export_roro', 'export_tanker', 'export_cargo', 'export'
]

# Group by timestamp and country, summing all metrics
grouped = df.groupby(['timestamp', 'country'])[numeric_cols].sum().reset_index()

# Pivot so each country becomes its own set of columns
pivoted = grouped.pivot(index='timestamp', columns='country', values=numeric_cols)

# Flatten multi-index columns
pivoted.columns = [f"{metric}_{country}" for metric, country in pivoted.columns]

# Reset index to make timestamp a column again
pivoted.reset_index(inplace=True)

# Save to CSV
pivoted.to_csv("data/countries_trading.csv", index=False)
print("✅ All rows saved to data/countries_trading.csv")


Fetched 1000 records so far...
Fetched 2000 records so far...
Fetched 3000 records so far...
Fetched 4000 records so far...
Fetched 5000 records so far...
Fetched 6000 records so far...
Fetched 7000 records so far...
Fetched 8000 records so far...
Fetched 9000 records so far...
Fetched 10000 records so far...
Fetched 11000 records so far...
Fetched 12000 records so far...
Fetched 13000 records so far...
Fetched 14000 records so far...
Fetched 15000 records so far...
Fetched 16000 records so far...
Fetched 17000 records so far...
Fetched 18000 records so far...
Fetched 19000 records so far...
Fetched 20000 records so far...
Fetched 21000 records so far...
Fetched 22000 records so far...
Fetched 23000 records so far...
Fetched 24000 records so far...
Fetched 25000 records so far...
Fetched 26000 records so far...
Fetched 27000 records so far...
Fetched 28000 records so far...
Fetched 29000 records so far...
Fetched 30000 records so far...
Fetched 31000 records so far...
Fetched 32000 rec